In [14]:
%pip -q install google-genai

In [19]:
# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai # Sua importação original
client = genai.Client() # Sua inicialização original

MODEL_ID = "gemini-2.0-flash" # Seu MODEL_ID original, com prefixo 'models/'

# Instalar Framework de agentes do Google
!pip install -q google-adk # Comentado, pois você já deve ter instalado

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types as genai_types_original # Sua importação e alias original
from datetime import date
import textwrap
from IPython.display import display, Markdown
# import requests # Não parece estar sendo usado
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# NENHUMA ALTERAÇÃO NESTA FUNÇÃO
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session_id_dinamico = f"session_{os.urandom(4).hex()}" # Usando ID dinâmico
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id=session_id_dinamico)
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = genai_types_original.Content(role="user", parts=[genai_types_original.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id=session_id_dinamico, new_message=content):
        if event.is_final_response():
          if hasattr(event, 'content') and event.content and hasattr(event.content, 'parts'):
            for part in event.content.parts:
              if hasattr(part, 'text') and part.text is not None:
                final_response += part.text
                # final_response += "\n" # Removido anteriormente, mantenho assim
    return final_response.strip() # .strip() para limpar espaços

# Função auxiliar para exibir texto formatado em Markdown no Colab
# NENHUMA ALTERAÇÃO NESTA FUNÇÃO
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [10]:
####################################################
# --- Agente 1: Coletor de DADOS de Doses --- #
####################################################
def agente_coletor_dados_doses(medicacao: str, especie: str, idade_fase_vida: str, peso_kg: float, data_de_hoje: str) -> str:
    coletor_dados_agent = Agent(
        name="agente_coletor_valores_doses_v3", # Novo nome
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um sistema de coleta de dados farmacológicos veterinários. Sua tarefa é encontrar informações específicas e retorná-las como UMA ÚNICA STRING DE TEXTO SIMPLES, com os valores separados pelo delimitador ';;'. NÃO inclua cabeçalhos de coluna. NÃO use formatação Markdown.

        Sua Tarefa Principal:
        Ao receber MEDICAÇÃO, ESPÉCIE, IDADE/FASE DE VIDA e PESO:
        1. Utilizar `google_search` para pesquisar doses da MEDICAÇÃO para a ESPÉCIE (foco: injetáveis) e segurança para a IDADE/FASE DE VIDA.
        2. Priorizar fontes confiáveis.
        3. Compilar os dados e apresentar a resposta EXCLUSIVAMENTE como UMA ÚNICA STRING, com os 7 valores na seguinte ordem, separados por ';;':
           1. Nome da Medicação
           2. Indicação Principal
           3. Apresentação (Concentração Injetável)
           4. Via de Administração
           5. Intervalo de Dose (mg/kg)
           6. Volume a Administrar (mL)
           7. Segurança para Idade/Fase de Vida (para a idade/fase de vida fornecida)

        Formato da Resposta (OBRIGATÓRIO - APENAS UMA STRING, VALORES SEPARADOS POR ';;'):
        [VALOR_MEDICAÇÃO];;[VALOR_INDICAÇÃO];;[VALOR_APRESENTAÇÃO];;[VALOR_VIA];;[VALOR_DOSE];;[VALOR_VOLUME];;[VALOR_SEGURANÇA_IDADE]

        Exemplo de Saída Esperada para Dipirona, Canina, Adulto 7 anos, 21kg:
        Dipirona;;Analgesia, antipirético e antiflogístico;;Dipirona Sódica 500 mg/mL;;IM, IV;;30 mg/kg;;1.26 mL;;Geralmente seguro para adultos, mas monitorar função renal e hepática.

        Instruções para Preenchimento dos Valores:
        - Se dados cruciais faltarem para um campo, use "Informação não encontrada" ou "N/A" como o valor para aquele campo.
        - Mantenha um tom técnico e preciso.
        - Sua resposta é APENAS a string com os valores separados por ';;'.
        """,
        description="Coleta dados de dose e retorna uma string com valores separados por ';;'.",
        tools=[google_search]
    )
    entrada_para_agente = f"Medicação: {medicacao}\nEspécie: {especie}\nIdade/Fase de Vida: {idade_fase_vida}\nPeso (kg): {peso_kg}\nData de Referência para pesquisa: {data_de_hoje}"
    valores_doses_str = call_agent(coletor_dados_agent, entrada_para_agente)
    return valores_doses_str.strip()

In [11]:
############################################################
# --- Agente 2: Coletor de DADOS de Farmaco/Toxico --- #
############################################################
def agente_coletor_dados_farm_tox(medicacao_principal: str, especie_paciente: str, idade_fase_vida_paciente: str, peso_paciente_kg: float) -> str:
    coletor_dados_ft_agent = Agent(
        name="agente_coletor_valores_farm_tox_v3", # Novo nome
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um sistema de coleta de dados farmacológicos e toxicológicos veterinários. Sua tarefa é encontrar informações específicas e retorná-las como UMA ÚNICA STRING DE TEXTO SIMPLES, com os valores separados pelo delimitador ';;'. NÃO inclua cabeçalhos de coluna. NÃO use formatação Markdown.

        Sua Tarefa Principal:
        A partir da MEDICAÇÃO PRINCIPAL, ESPÉCIE, IDADE/FASE DE VIDA:
        1. Utilizar `google_search` para pesquisar informações detalhadas SOBRE A MEDICAÇÃO PRINCIPAL para a ESPÉCIE, considerando a IDADE/FASE DE VIDA.
        2. Focar em: Dose Tóxica, Efeitos Adversos Relevantes, Tratamento da Intoxicação, Interações Medicamentosas de Risco, Sinergismo Farmacológico Benéfico.
        3. Priorizar fontes confiáveis.
        4. Compilar os dados e apresentar a resposta EXCLUSIVAMENTE como UMA ÚNICA STRING, com os 6 valores na seguinte ordem, separados por ';;':
           1. Nome da Medicação
           2. Dose Tóxica (Espécie, Idade se relevante)
           3. Efeitos Adversos Relevantes (Considerar Idade)
           4. Tratamento Intoxicação
           5. Interações Medicamentosas de Risco
           6. Interações Sinergéticas Benéficas

        Formato da Resposta (OBRIGATÓRIO - APENAS UMA STRING, VALORES SEPARADOS POR ';;'):
        [VALOR_MEDICAÇÃO];;[VALOR_DOSE_TOXICA];;[VALOR_EFEITOS_ADVERSOS];;[VALOR_TRATAMENTO];;[VALOR_INTERACOES_RISCO];;[VALOR_SINERGIA]

        Exemplo de Saída Esperada para Dipirona:
        Dipirona;;>600 mg/kg (canina);;Êmese, diarreia, gastrite (raro: leucopenia);;Suporte hidroeletrolítico, carvão ativado;;Com AINEs: ↑risco TGI;;Com Tramadol: analgesia potencializada

        Instruções para Preenchimento dos Valores:
        - Se informação não encontrada para um campo, use "Dados insuficientes" ou "N/A" como o valor para aquele campo.
        - Use terminologia técnica precisa.
        - Sua resposta é APENAS a string com os valores separados por ';;'.
        """,
        description="Coleta dados farmaco/toxico e retorna uma string com valores separados por ';;'.",
        tools=[google_search]
    )
    entrada_para_agente = f"Medicação Principal: {medicacao_principal}\nEspécie do Paciente: {especie_paciente}\nIdade/Fase de Vida do Paciente: {idade_fase_vida_paciente}\nPeso do Paciente (kg): {peso_paciente_kg}"
    valores_farm_tox_str = call_agent(coletor_dados_ft_agent, entrada_para_agente)
    return valores_farm_tox_str.strip()

In [12]:
#############################################################
# --- Agente 3: Conselheiro Clínico Farmacológico --- #
#############################################################
def agente_conselheiro_clinico(
    medicacoes_consultadas_ctx: list,
    especie_paciente_ctx: str,
    idade_fase_vida_ctx: str,
    peso_paciente_kg_ctx: float,
    tabela_doses_md_str: str,
    tabela_farm_tox_md_str: str
) -> str:
    conselheiro_agent = Agent(
        name="agente_analista_farmacologico_ia", # Nome atualizado para refletir a nova persona
        model=MODEL_ID,
        instruction=f"""
        Sua Persona e Missão:
        Você é um **assistente de IA especializado em análise farmacológica e toxicológica veterinária**. Sua função é atuar como uma ferramenta de suporte à decisão clínica para médicos veterinários, fornecendo uma análise técnica dos dados apresentados. Sua linguagem deve ser precisa, objetiva e utilizar TERMINOLOGIA VETERINÁRIA TÉCNICA quando apropriado para descrever os achados da análise. **Você NÃO é um médico veterinário, mas sim uma ferramenta de análise.**

        Sua tarefa é analisar criticamente as informações farmacológicas apresentadas em DUAS tabelas (Doses e Farmaco/Toxico) para a(s) medicação(ões) [{', '.join(medicacoes_consultadas_ctx) if isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx}] em um(a) paciente da espécie "[{especie_paciente_ctx}]", com IDADE/FASE DE VIDA de "[{idade_fase_vida_ctx}]" e peso aproximado de "[{peso_paciente_kg_ctx}] kg".
        Com base nessa análise, seu objetivo é fornecer **uma análise técnica dos dados, destacar pontos de atenção críticos (especialmente relacionados à idade/fase de vida e possíveis interações entre múltiplas drogas, se aplicável) e identificar potenciais otimizações terapêuticas com base nos dados fornecidos, apresentando isso como um relatório analítico.**

        As tabelas recebidas contêm:
        1.  **Tabela de Informações de Dose:** Medicação, Indicação, apresentação, via, dose, volume e avaliação de segurança para idade/fase de vida.
        2.  **Tabela de Informações Toxicológicas e Farmacodinâmicas:** Medicação, Dose tóxica, efeitos adversos, tratamento intoxicação, interações medicamentosas de risco e sinergismo benéfico.

        Seu Foco ao Revisar (para formular seu relatório analítico):
        1.  **Consistência e Potenciais Riscos (Considerando Idade):** As informações para cada droga nas tabelas são consistentes com dados farmacológicos conhecidos? Há riscos evidentes baseados na idade/fase de vida, dose tóxica, ou efeitos adversos listados?
        2.  **Interações Medicamentosas (se múltiplas drogas):** Com base nas informações das tabelas ("Interações Medicamentosas de Risco" e "Interações Sinergéticas Benéficas"), há interações significativas ou potenciais entre as medicações consultadas?
        3.  **Relevância Clínica e Pontos de Alerta (Foco na Idade e Protocolo Geral):** Com base nas informações de "Segurança para Idade/Fase de Vida" e "Efeitos Adversos Relevantes", existem alertas específicos para otimizar a segurança terapêutica? A dose e volume calculados parecem apropriados ou levantam questões?
        4.  **Gestão de Riscos e Segurança Farmacoterapêutica (baseado nos dados):** Quais riscos específicos são identificados nas tabelas? O "Tratamento Intoxicação" listado é relevante para os efeitos adversos?

        Formato da SUA Resposta (RELATÓRIO ANALÍTICO):
        - **Inicie SEMPRE sua resposta com:** "Como um assistente de IA especializado, segue a análise técnica dos dados fornecidos:"
        - Se, com base na análise dos dados fornecidos, tudo estiver aparentemente adequado:
          "ANÁLISE FARMACOLÓGICA PRELIMINAR: Os dados para [{', '.join(medicacoes_consultadas_ctx) if isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx}] em {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg), conforme apresentados nas tabelas, parecem consistentes e sem alertas críticos imediatos. Recomenda-se a revisão e validação por um médico veterinário."

        - Caso identifique pontos de atenção, inconsistências nos dados fornecidos, ou necessite de detalhamento com base na análise das tabelas:
            Formule um relatório técnico. Se múltiplas drogas, aborde interações com base nas informações das tabelas.
            Exemplo para uma droga:
            "Analisando os dados de {medicacoes_consultadas_ctx if not isinstance(medicacoes_consultadas_ctx, list) else medicacoes_consultadas_ctx[0]} para {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg) conforme tabelas:\\n- **Ponto de Análise (Dose/Segurança):** [Detalhe técnico da análise dos dados de dose, volume, segurança para idade].\\n- **Ponto de Análise (Farmaco/Toxico):** [Detalhe técnico da análise dos dados de efeitos adversos, interações listadas]."

            Exemplo para múltiplas drogas:
            "Analisando o protocolo com base nos dados de [{', '.join(medicacoes_consultadas_ctx)}] para {especie_paciente_ctx} ({idade_fase_vida_ctx}, {peso_paciente_kg_ctx} kg) conforme tabelas:\\n- **{medicacoes_consultadas_ctx[0] if len(medicacoes_consultadas_ctx)>0 else 'Medicação 1'} (análise dos dados):** [Análise dos dados específicos para esta medicação das tabelas].\\n- **{medicacoes_consultadas_ctx[1] if len(medicacoes_consultadas_ctx)>1 else 'Medicação 2'} (análise dos dados):** [Análise dos dados específicos para esta medicação das tabelas].\\n- **Interações Potenciais (com base nas tabelas):** [Detalhe da análise das interações listadas nas tabelas entre {medicacoes_consultadas_ctx[0] if len(medicacoes_consultadas_ctx)>0 else ''} e {medicacoes_consultadas_ctx[1] if len(medicacoes_consultadas_ctx)>1 else ''}]."

        - **Finalize SEMPRE sua resposta, em uma nova linha separada, com:**
          "Lembre-se: Este é um relatório analítico gerado por uma ferramenta de IA e não substitui o julgamento clínico, a experiência profissional e a responsabilidade final de um médico veterinário qualificado. As informações aqui contidas devem ser cuidadosamente avaliadas e validadas pelo profissional responsável antes de qualquer decisão clínica."

        Utilize `google_search` APENAS SE ESTRITAMENTE NECESSÁRIO para verificar alguma inconsistência FLAGRANTE nos dados das tabelas fornecidas, mas sua análise primária deve ser sobre os dados recebidos.
        Sua resposta NÃO é uma tabela, mas sim um relatório técnico dissertativo.
        """,
        description="Fornece análise técnica farmacológica baseada em dados, como um assistente de IA.",
        tools=[google_search]
    )

    meds_str_param = medicacoes_consultadas_ctx
    if isinstance(medicacoes_consultadas_ctx, list):
        meds_str_param = ', '.join(medicacoes_consultadas_ctx)

    entrada_para_agente = (
        f"DADOS PARA ANÁLISE E PARECER FARMACOLÓGICO ASSISTIDO POR IA:\n"
        f"Medicação(ões) Principal(ais) para análise: {meds_str_param}\n"
        f"Espécie do Paciente: {especie_paciente_ctx}\n"
        f"Idade/Fase de Vida do Paciente: {idade_fase_vida_ctx}\n"
        f"Peso do Paciente (kg): {peso_paciente_kg_ctx}\n\n"
        f"TABELA 1 RECEBIDA (INFORMAÇÕES DE DOSE - Markdown):\n{tabela_doses_md_str}\n\n"
        f"TABELA 2 RECEBIDA (INFORMAÇÕES TOXICOLÓGICAS E FARMACODINÂMICAS - Markdown):\n{tabela_farm_tox_md_str}\n\n"
        f"Por favor, analise criticamente estas informações conforme suas instruções e forneça seu relatório analítico."
    )
    feedback_do_conselheiro = call_agent(conselheiro_agent, entrada_para_agente)
    return feedback_do_conselheiro

In [20]:
# --- Fluxo Principal de Execução --- #
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("Iniciando o Sistema de Consulta Farmacológica Veterinária ")
print("="*60)

try:
    especie_ctx_global = input("🐾 Digite a ESPÉCIE do paciente (ex: Canina, Felina): ").strip()
    idade_input_str_global = input("📅 Digite a IDADE do paciente (ex: 2 meses, 1 ano, adulto): ").strip()
    peso_input_str_global = input("⚖️ Digite o PESO do paciente em kg (ex: 21): ").strip()
    medicacoes_input_str = input("💊 Digite a(s) MEDICAÇÃO(ÕES) (separadas por vírgula): ").strip()

    if not especie_ctx_global or not idade_input_str_global or not peso_input_str_global or not medicacoes_input_str:
        raise ValueError("Um ou mais campos de entrada estão vazios.")

    if "kg" in peso_input_str_global.lower():
        peso_kg_ctx_global = float(peso_input_str_global.lower().replace("kg", "").strip())
    else:
        peso_kg_ctx_global = float(peso_input_str_global.strip())
    idade_fase_vida_ctx_global = idade_input_str_global
    lista_medicacoes_consultadas = [med.strip() for med in medicacoes_input_str.split(',') if med.strip()]
    if not lista_medicacoes_consultadas:
        raise ValueError("Nenhuma medicação fornecida.")

    print("\n======================================================")


    # Estruturas para armazenar as linhas de dados FORMATADAS EM MARKDOWN para as tabelas finais
    linhas_markdown_tabela_doses = []
    linhas_markdown_tabela_farm_tox = []

    # Cabeçalhos e separadores são definidos uma vez
    cabecalho_tabela_doses = "| Medicação | Indicação Principal | Apresentação (Concentração Injetável) | Via de Administração | Intervalo de Dose (mg/kg) | Volume a Administrar (mL) | Segurança para Idade/Fase de Vida |"
    separador_tabela_doses = "| :-------- | :------------------ | :------------------------------------ | :------------------- | :------------------------ | :------------------------ | :------------------------------------ |"

    cabecalho_tabela_farm_tox = "| Medicação | Dose Tóxica (Espécie, Idade relevante) | Efeitos Adversos Relevantes (Considerar Idade) | Tratamento Intoxicação | Interações Medicamentosas de Risco | Interações Sinergéticas Benéficas |"
    separador_tabela_farm_tox = "| :-------- | :----------------------------------------- | :--------------------------------------------- | :----------------------- | :------------------------------------ | :--------------------------------------- |"

    linhas_markdown_tabela_doses.append(cabecalho_tabela_doses)
    linhas_markdown_tabela_doses.append(separador_tabela_doses)
    linhas_markdown_tabela_farm_tox.append(cabecalho_tabela_farm_tox)
    linhas_markdown_tabela_farm_tox.append(separador_tabela_farm_tox)

    houve_erro_geral = False

    print(f"\n⏳ Processando informações... Por favor, aguarde.")

    for medicacao_individual_ctx in lista_medicacoes_consultadas:
        print(f"  Coletando dados para: {medicacao_individual_ctx}...") # Feedback mais discreto
        erro_nesta_droga = False

        # --- Agente 1: Coletor de DADOS de Doses ---
        valores_doses_str = agente_coletor_dados_doses( # Retorna string com valores separados por ;;
            medicacao_individual_ctx, especie_ctx_global, idade_fase_vida_ctx_global, peso_kg_ctx_global, data_de_hoje
        )
        if valores_doses_str.strip().startswith("ERRO"):
            print(f"    ERRO do Agente 1 para {medicacao_individual_ctx}: {valores_doses_str}")
            linhas_markdown_tabela_doses.append(f"| {medicacao_individual_ctx} | ERRO AO OBTER DADOS DE DOSE | - | - | - | - | - |")
            erro_nesta_droga = True
            houve_erro_geral = True
        else:
            valores_t1 = valores_doses_str.split(';;')
            if len(valores_t1) == 7: # Espera 7 valores do Agente 1
                linhas_markdown_tabela_doses.append(f"| {valores_t1[0].strip()} | {valores_t1[1].strip()} | {valores_t1[2].strip()} | {valores_t1[3].strip()} | {valores_t1[4].strip()} | {valores_t1[5].strip()} | {valores_t1[6].strip()} |")
            else:
                print(f"    AVISO: Agente 1 para {medicacao_individual_ctx} retornou formato inesperado: {valores_doses_str}")
                linhas_markdown_tabela_doses.append(f"| {medicacao_individual_ctx} | ERRO DE FORMATO (AG1) | - | - | - | - | - |")
                erro_nesta_droga = True
                houve_erro_geral = True

        # --- Agente 2: Coletor de DADOS de Farmaco/Toxico ---
        if not erro_nesta_droga:
            valores_farm_tox_str = agente_coletor_dados_farm_tox( # Retorna string com valores separados por ;;
                medicacao_individual_ctx, especie_ctx_global, idade_fase_vida_ctx_global, peso_kg_ctx_global
            )
            if valores_farm_tox_str.strip().startswith("ERRO"):
                print(f"    ERRO do Agente 2 para {medicacao_individual_ctx}: {valores_farm_tox_str}")
                linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | ERRO AO OBTER DADOS FARMACO/TOXICO | - | - | - | - |")
                houve_erro_geral = True
            else:
                valores_t2 = valores_farm_tox_str.split(';;')
                if len(valores_t2) == 6: # Espera 6 valores do Agente 2
                    linhas_markdown_tabela_farm_tox.append(f"| {valores_t2[0].strip()} | {valores_t2[1].strip()} | {valores_t2[2].strip()} | {valores_t2[3].strip()} | {valores_t2[4].strip()} | {valores_t2[5].strip()} |")
                else:
                    print(f"    AVISO: Agente 2 para {medicacao_individual_ctx} retornou formato inesperado: {valores_farm_tox_str}")
                    linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | ERRO DE FORMATO (AG2) | - | - | - | - |")
                    houve_erro_geral = True
        else:
             linhas_markdown_tabela_farm_tox.append(f"| {medicacao_individual_ctx} | DADOS NÃO COLETADOS (ERRO AG1) | - | - | - | - |")

    # --- Exibição das Tabelas Consolidadas ---
    print("\n\n======================================================")
    print("--- 📄 RELATÓRIO FARMACOLÓGICO CONSOLIDADO ---")
    print("======================================================")

    contexto_paciente_str_display = f"**Medicações Consultadas:** {', '.join(lista_medicacoes_consultadas)}\n**Espécie:** {especie_ctx_global} | **Idade/Fase de Vida:** {idade_fase_vida_ctx_global} | **Peso:** {peso_kg_ctx_global} kg\n"
    display(Markdown(contexto_paciente_str_display))

    print("\n**Tabela 1: Informações de Dose**")
    tabela_doses_final_str = "\n".join(linhas_markdown_tabela_doses)
    display(Markdown(tabela_doses_final_str))

    print("\n**Tabela 2: Informações Toxicológicas e Farmacodinâmicas**")
    tabela_farm_tox_final_str = "\n".join(linhas_markdown_tabela_farm_tox)
    display(Markdown(tabela_farm_tox_final_str))
    print("--------------------------------------------------------------")

    # --- Agente 3 (Conselheiro Clínico - antigo Agente 4) ---
    if not houve_erro_geral:
        revisar = input(f"\n❓ Deseja o parecer do Conselheiro Clínico sobre o relatório consolidado? (s/n): ").strip().lower()
        if revisar == 's':
            parecer_conselheiro = agente_conselheiro_clinico(
                lista_medicacoes_consultadas,
                especie_ctx_global,
                idade_fase_vida_ctx_global,
                peso_kg_ctx_global,
                tabela_doses_final_str,
                tabela_farm_tox_final_str
            )
            print(f"\n--- 🧐 Parecer do Agente 3 (Conselheiro Clínico) ---")
            display(Markdown(parecer_conselheiro))
            print("--------------------------------------------------------------")
        else:
            print(f"\nℹ️ Parecer do Conselheiro Clínico pulado.")
    else:
        print("\n⚠️ Conselheiro Clínico (Agente 3) não executado devido a erros na coleta de dados.")

    print("\n✅✅✅ Todas as consultas foram processadas! ✅✅✅")

except ValueError as e:
    print(f"❌ Erro na entrada de dados: {e}. Por favor, reinicie e tente novamente.")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado durante a execução: {e} (Tipo: {type(e).__name__})")
    traceback.print_exc()

Iniciando o Sistema de Consulta Farmacológica Veterinária 
🐾 Digite a ESPÉCIE do paciente (ex: Canina, Felina): Canino
📅 Digite a IDADE do paciente (ex: 2 meses, 1 ano, adulto): 3 anos
⚖️ Digite o PESO do paciente em kg (ex: 21): 10
💊 Digite a(s) MEDICAÇÃO(ÕES) (separadas por vírgula): Dipirona, Tramadol


⏳ Processando informações... Por favor, aguarde.
  Coletando dados para: Dipirona...
  Coletando dados para: Tramadol...


--- 📄 RELATÓRIO FARMACOLÓGICO CONSOLIDADO ---


**Medicações Consultadas:** Dipirona, Tramadol
**Espécie:** Canino | **Idade/Fase de Vida:** 3 anos | **Peso:** 10.0 kg



**Tabela 1: Informações de Dose**


| Medicação | Indicação Principal | Apresentação (Concentração Injetável) | Via de Administração | Intervalo de Dose (mg/kg) | Volume a Administrar (mL) | Segurança para Idade/Fase de Vida |
| :-------- | :------------------ | :------------------------------------ | :------------------- | :------------------------ | :------------------------ | :------------------------------------ |
| Dipirona | Analgesia, antipirético e antiflogístico | Dipirona Sódica 500 mg/mL | IM, IV | 30 mg/kg | 0.6 mL | Geralmente seguro, mas usar com cautela em animais com problemas renais ou hepáticos. |
| Tramadol | Alívio da dor moderada | Solução injetável 50 mg/mL (2%) | IM, IV | 2-4 mg/kg a cada 6-8 horas | 0.04-0.08 mL/kg | Administrar com precaução em cães com insuficiência renal ou hepática. |


**Tabela 2: Informações Toxicológicas e Farmacodinâmicas**


| Medicação | Dose Tóxica (Espécie, Idade relevante) | Efeitos Adversos Relevantes (Considerar Idade) | Tratamento Intoxicação | Interações Medicamentosas de Risco | Interações Sinergéticas Benéficas |
| :-------- | :----------------------------------------- | :--------------------------------------------- | :----------------------- | :------------------------------------ | :--------------------------------------- |
| Dipirona | >600 mg/kg (canina) | Vômito, diarreia, gastrite (raro: leucopenia, hemorragia) | Suporte hidroeletrolítico, carvão ativado | Com AINEs: ↑risco TGI, Com Clorpromazina: hipotermia | Com Tramadol: analgesia potencializada, Com inibidores da ECA, bloqueadores adrenérgicos e bloqueadores dos canais de cálcio: interação segura |
| Tramadol | 1-8mg/kg (pode variar) | Sedação, náuseas, vômitos, constipação, agitação | Lavagem gástrica (se consciente), naloxona (depressão respiratória), diazepam (convulsões) | Depressores do SNC, inibidores da MAO, ISRS, antidepressivos tricíclicos, fármacos que diminuem o limiar convulsivo | AINEs: sinergia analgésica |

--------------------------------------------------------------

❓ Deseja o parecer do Conselheiro Clínico sobre o relatório consolidado? (s/n): s

--- 🧐 Parecer do Agente 3 (Conselheiro Clínico) ---


Como um assistente de IA especializado, segue a análise técnica dos dados fornecidos:

Analisando o protocolo com base nos dados de [Dipirona, Tramadol] para Canino (3 anos, 10.0 kg) conforme tabelas:
- **Dipirona (análise dos dados):** A dose de Dipirona (30 mg/kg) resulta em um volume de 0.6 mL, o que é consistente com a apresentação injetável de 500 mg/mL. A segurança para idade/fase de vida indica cautela em animais com problemas renais ou hepáticos, o que requer avaliação prévia da função renal e hepática do paciente. A dose tóxica (>600 mg/kg) está bem acima da dose terapêutica, mas a ocorrência de vômito, diarreia e gastrite, embora rara, deve ser monitorada.
- **Tramadol (análise dos dados):** A dose de Tramadol (2-4 mg/kg) resulta em um volume de 0.04-0.08 mL/kg, o que é consistente com a apresentação injetável de 50 mg/mL. A administração com precaução em cães com insuficiência renal ou hepática reforça a necessidade de avaliação prévia. Os efeitos adversos relevantes incluem sedação, náuseas, vômitos e constipação, que devem ser monitorados.
- **Interações Potenciais (com base nas tabelas):** A Dipirona apresenta interação de risco com AINEs, aumentando o risco de problemas gastrointestinais (TGI), o que não é relevante neste caso, já que o protocolo inclui Tramadol e Dipirona. A interação sinergética benéfica entre Dipirona e Tramadol para analgesia potencializada é favorável. O Tramadol apresenta interações de risco com depressores do SNC, inibidores da MAO, ISRS, antidepressivos tricíclicos e fármacos que diminuem o limiar convulsivo, o que requer uma revisão detalhada da história medicamentosa do paciente para evitar combinações perigosas.

Lembre-se: Este é um relatório analítico gerado por uma ferramenta de IA e não substitui o julgamento clínico, a experiência profissional e a responsabilidade final de um médico veterinário qualificado. As informações aqui contidas devem ser cuidadosamente avaliadas e validadas pelo profissional responsável antes de qualquer decisão clínica.

--------------------------------------------------------------

✅✅✅ Todas as consultas foram processadas! ✅✅✅
